<a href="https://colab.research.google.com/github/olwynodpatterson/FYP/blob/main/10kForms_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping 10k fillings

In [10]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random

from google.colab import drive

# Function to retrieve a list of tickers from a given URL
def get_ticker_list(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        tickers = response.text.splitlines()
        return tickers
    else:
        # Raise an exception if the request failed
        raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")

# Function to choose a random CIK (Central Index Key) from the list of tickers
def choose_random_cik(tickers):
    # Choose a random ticker and associated CIK
    ticker, cik = random.choice(tickers).split('\t')
    return cik

# Function to get the URLs of 10-K filings for a given CIK
def get_10k_filing_urls(cik):
    time.sleep(3) # Delay to avoid overwhelming the server
    # Fetch the URLs for 10-K filings
    filings_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K&dateb=&owner=exclude&count=10"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(filings_url, headers=headers)
    # Check response status
    if response.status_code != 200:
        print("Error fetching filings:", response.status_code)
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='tableFile2')
    if not table:
        return None

    # Parse the table to get the filing links
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            documents_page_link = 'https://www.sec.gov' + cols[1].a['href']
            submission_text_file_link = documents_page_link.replace('-index.htm', '.txt')
            return submission_text_file_link  # Return the first link found

    return None

def dehtml(html_content):
    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text()

    return text

# Mount Google Drive
drive.mount('/content/drive')

# Function to download a file from a given URL
def download_file(url, cik, folder='/content/drive/My Drive/10k_filings'):
    # Define the Google Drive folder path
    drive_folder_path = os.path.join(folder, cik)

    # Create the folder if it doesn't exist
    if not os.path.exists(drive_folder_path):
        os.makedirs(drive_folder_path)

    # Set the file path using the CIK
    html_file_path = os.path.join(folder, f'{cik}_10k_form.html')
    text_file_path = os.path.join(folder, f'{cik}_10k_form.txt')

    # Make a request to download the file
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(html_file_path, 'wb') as html_file:
            for chunk in response.iter_content(chunk_size=8192):
                html_file.write(chunk)
        print(f"Downloaded HTML: {html_file_path}")

        # Read the HTML content
        with open(html_file_path, 'r', encoding='utf-8') as html_file:
            html_content = html_file.read()

        # Convert HTML to plain text
        plain_text = dehtml(html_content)

        # Save the plain text
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(plain_text)
        print(f"Converted to Text: {text_file_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# Main function to drive the program
def main():
    url = 'https://www.sec.gov/include/ticker.txt'
    tickers = get_ticker_list(url)
    i=0
    while i <1:
        cik =  '1332349'#choose_random_cik(tickers)
        print(f"Selected CIK: {cik}")
        filing_url = get_10k_filing_urls(cik)
        print(f"Filing URL: {filing_url}")
        if filing_url:
            download_file(filing_url, cik)
            time.sleep(1)  # Respectful delay between requests
            i += 1
        else:
            print(f"No 10-K filings found for CIK {cik}.")

if __name__ == "__main__":
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Selected CIK: 1332349
Filing URL: https://www.sec.gov/Archives/edgar/data/1332349/000133234923000047/0001332349-23-000047.txt
Downloaded HTML: /content/drive/My Drive/10k_filings/1332349_10k_form.html
https://www.sec.gov/Archives/edgar/data/1332349/000133234923000047/0001332349-23-000047.txt
Converted to Text: /content/drive/My Drive/10k_filings/1332349_10k_form.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Breaking the Data into sections

This project will focus on the Revenue section of the 10-k so I must break down the data to find these sections.


In [22]:
#This is rough wrk for one file
from bs4 import BeautifulSoup
import pandas as pd
import re

# Path to your downloaded HTML file
file_path = '/content/drive/My Drive/10k_filings/1332349_10k_form.html'

# Open and read the file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table
#part5 = soup.find_all('span', string='PART IV')[1]
#print(part5)

# If you need to also ensure the <a> tag has an 'href' attribute (i.e., it is a link)
#balance_sheet = soup.find_all('a', href=True, string=lambda text: text and 'Consolidated Balance Sheet' in text)[0]
#print(balance_sheet)

# Find the <a> tag
a_tag = soup.find('a', string=lambda text: text and 'Consolidated Balance Sheet' in text)

if a_tag and a_tag.has_attr('href'):
    href = a_tag['href']

    # Check if it's an internal link
    if href.startswith('#'):
        target_id = href[1:]  # Remove the '#' at the beginning
        target_element = soup.find(id=target_id)

        if target_element:
            print("Found the target element:", target_element)

            # Assuming 'target_element' is the element you've located
            next_table = target_element.find_next('table')
            if next_table:
                print("Found the next table:", next_table)
                # Let's say 'next_table' is the BeautifulSoup object for the table you've found
                html_string = str(next_table)

                # Use Pandas to read the HTML string; this returns a list of DataFrames
                dfs = pd.read_html(html_string)

                # Assuming there is only one table, or you're interested in the first one
                table_df = dfs[0]
                print(table_df)
                # Specify the file path and name for your CSV file
                csv_file_path = '/content/drive/My Drive/10k_filings/1332349_balance_sheet.csv'

                # Export the DataFrame to a CSV file
                table_df.to_csv(csv_file_path, index=False)  # Set index=False if you don't want to include the DataFrame index in the file

            else:
                print("No next table found.")

        else:
            print("Target element not found.")
    else:
        print("The link is not an internal link.")



Found the target element: <div id="i47886ca125c7405582a8d73769ffab87_151"></div>
Found the next table: <table style="border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:100.000%"><tr><td style="width:1.0%"></td><td style="width:72.730%"></td><td style="width:0.1%"></td><td style="width:1.0%"></td><td style="width:11.619%"></td><td style="width:0.1%"></td><td style="width:0.1%"></td><td style="width:0.530%"></td><td style="width:0.1%"></td><td style="width:1.0%"></td><td style="width:11.621%"></td><td style="width:0.1%"></td></tr><tr><td colspan="3" style="padding:0 1pt"></td><td colspan="9" style="padding:2px 1pt;text-align:center;vertical-align:bottom"><span style="color:#000000;font-family:'Times New Roman',sans-serif;font-size:9.5pt;font-weight:700;line-height:100%">December 31,</span></td></tr><tr><td colspan="3" style="padding:0 1pt"></td><td colspan="3" style="border-top:2pt solid #000000;padding:2px 1pt;text-align:center;vertical-alig